In [ ]:
!git clone https://github.com/Fonsydo/Proyecto_IA.git

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
credits = pd.read_csv('/content/Proyecto_IA/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/content/Proyecto_IA/the-movies-dataset/keywords.csv')
movies = pd.read_csv('/content/Proyecto_IA/the-movies-dataset/movies_metadata.csv', low_memory=False).\
                        drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1)

movies['id'] = movies['id'].apply(pd.to_numeric, errors='coerce')
movies.dropna(inplace=True)
movies['id'] = movies['id'].astype('int64')

df = movies.merge(keywords, on='id'). merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)
df.head(5)

In [ ]:
def get_text(text, obj='name'):
    text = eval(text)
    
    if len(text) == 1:
        for i in text:
            return i[obj]
    else:
        s = []
        for i in text:
            s.append(i[obj])
        return ', '.join(s)

df['genres'] = df['genres'].apply(get_text)
df['production_companies'] = df['production_companies'].apply(get_text)
df['production_countries'] = df['production_countries'].apply(get_text)
df['crew'] = df['crew'].apply(get_text)
df['spoken_languages'] = df['spoken_languages'].apply(get_text)
df['keywords'] = df['keywords'].apply(get_text)

df['characters'] = df['cast'].apply(get_text, obj='character')
df['actors'] = df['cast'].apply(get_text)

df.drop('cast', axis=1, inplace=True)
df = df[~df['original_title'].duplicated()]
df = df.reset_index(drop=True)

In [ ]:
ratings_df = pd.read_csv('the-movies-dataset/ratings.csv')

ratings_df['date'] = ratings_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings_df.drop('timestamp', axis=1, inplace=True)

ratings_df = ratings_df.merge(df[['id', 'original_title', 'genres', 'overview']], left_on='movieId',right_on='id', how='left')
ratings_df = ratings_df[~ratings_df['id'].isna()]
ratings_df.drop('id', axis=1, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)
ratings_df.head(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
mvoie_encoder = LabelEncoder()

ratings_df['userId'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['movieId'] = mvoie_encoder.fit_transform(ratings_df['movieId'])

ratings_df.head(5)

In [ ]:
min_movie_ratings = 10000
filter_movies = (ratings_df.groupby('movieId')['rating'].sum()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

min_user_ratings = 200
filter_users = (ratings_df.groupby('userId')['rating'].sum()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

df_filterd = ratings_df[(ratings_df['movieId'].isin(filter_movies)) & (ratings_df['userId'].isin(filter_users))]

print('Shape User-Ratings unfiltered:\t{}'.format(ratings_df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_filterd, test_size=0.3)
train_df.shape, test_df.shape

In [ ]:
def get_model1(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT):
    from keras.layers import Input, Embedding, Dense, Dropout, Dot
    from keras.models import Model

    movie_input = Input(shape=(1,), name='movie_id')

    movie_emb = Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_MOVIES, input_length=ROW_COUNT, name='movie_emb')(movie_input)
    
    user_input = Input(shape=(1,), name='user_id')

    user_emb = Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_USERS, input_length=ROW_COUNT, name='user_emb')(user_input)
    
    merged = Dot(name = 'dot_product', normalize = True, axes = 2)([movie_emb, user_emb])
    merged = Dropout(0.2)(merged)
    
    dense_1 = Dense(70,name='FullyConnected-1')(merged)
    dropout_1 = Dropout(0.2,name='Dropout_1')(dense_1)

    dense_2 = Dense(50,name='FullyConnected-2')(dropout_1)
    dropout_2 = Dropout(0.2,name='Dropout_2')(dense_2)

    dense_3 = Dense(20,name='FullyConnected-3')(dropout_2)
    dropout_3 = Dropout(0.2,name='Dropout_3')(dense_3)

    dense_4 = Dense(10,name='FullyConnected-4', activation='relu')(dropout_3)

    result = Dense(1, name='result', activation="relu") (dense_4)
    model = Model([movie_input, user_input], result)
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
ROW_COUNT = train_df.shape[0]
EMBEDDING_SIZE = 10
NUM_USERS = ratings_df['userId'].nunique()
NUM_MOVIES = ratings_df['movieId'].nunique()
UNIQUE_MOVIE_IDS = ratings_df['movieId'].unique()
MOVIE_METADATA = ratings_df.groupby('movieId', as_index=False)[['original_title', 'genres', 'overview']].first()

NUM_USERS, NUM_MOVIES, ROW_COUNT

In [ ]:
model1 = get_model1(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT)

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model1, show_shapes=True, show_layer_names=True)

In [ ]:
history1 = model1.fit([train_df['movieId'], train_df['userId']],
          train_df['rating'],
          batch_size=2048, 
          epochs=20,
          validation_split=0.1,
          shuffle=True)

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model1.predict([test_df['movieId'], test_df['userId']]).reshape(-1)
y_true = test_df['rating'].values

#  Compute RMSE
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Matrix-Factorization: {:.4f} RMSE'.format(rmse))

In [ ]:
def plot_loss(history):
    import matplotlib.pyplot as plt

    training_loss = history.history['loss']
    test_loss = history.history['val_loss']

    epoch_count = range(1, len(training_loss) + 1)

    plt.figure(figsize = (8,4))
    plt.grid()
    plt.plot(epoch_count, training_loss, '--', color='#06837f')
    plt.plot(epoch_count, test_loss, '-', color='#fed811')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
plot_loss(history1)

In [ ]:
tags = [i.split(", ") for i in df['genres'].unique()]
GENRES = list(set([i for lst in tags for i in lst]))
GENRES.remove('')
print(GENRES)

In [ ]:
for col in GENRES:
    train_df[col] = train_df["genres"].apply(lambda x: 1 if col in x else 0)
    test_df[col] = test_df["genres"].apply(lambda x: 1 if col in x else 0)
test_df.head()

In [ ]:
def get_model2(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT, NUM_GENRES):
    from keras.layers import Input, Embedding, Dense, Dropout, Dot, Reshape, Concatenate
    from keras.models import Model

    movie_input = Input(shape=(1,), name='movie_id')

    movie_emb = Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_MOVIES, input_length=ROW_COUNT, name='movie_emb')(movie_input)

    movie_res = Reshape(target_shape=(EMBEDDING_SIZE,), name='movie_res')(movie_emb)

    user_input = Input(shape=(1,), name='user_id')

    user_emb = Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_USERS, input_length=ROW_COUNT, name='user_emb')(user_input)

    user_res = Reshape(target_shape=(EMBEDDING_SIZE,), name='user_res')(user_emb)
    
    merged = Dot(name = 'dot_product', normalize = True, axes = 1)([movie_res, user_res])

    #############################
    genres_input = Input(shape=(NUM_GENRES,),name='genres')
    dense_gen_1 = Dense(70,name='FullyConnected-gen_1')(genres_input)
    dropout_gen_1 = Dropout(0.2,name='Dropout_gen_1')(dense_gen_1)

    dense_gen_2 = Dense(50,name='FullyConnected-gen_2')(dropout_gen_1)
    dropout_gen_2 = Dropout(0.2,name='Dropout_gen_2')(dense_gen_2)

    dense_gen_3 = Dense(20,name='FullyConnected-gen_3')(dropout_gen_2)
    dropout_gen_3 = Dropout(0.2,name='Dropout_gen_3')(dense_gen_3)

    dense_gen_4 = Dense(10,name='FullyConnected-gen_4', activation='relu')(dropout_gen_3)
    #############################

    merged = Dropout(0.2)(merged)
    
    dense_1 = Dense(70,name='FullyConnected-1')(merged)
    dropout_1 = Dropout(0.2,name='Dropout_1')(dense_1)

    dense_2 = Dense(50,name='FullyConnected-2')(dropout_1)
    dropout_2 = Dropout(0.2,name='Dropout_2')(dense_2)

    dense_3 = Dense(20,name='FullyConnected-3')(dropout_2)
    dropout_3 = Dropout(0.2,name='Dropout_3')(dense_3)

    dense_4 = Dense(10,name='FullyConnected-4', activation='relu')(dropout_3)

    concat = Concatenate()([dense_4, dense_gen_4])

    result = Dense(1, name='result', activation="relu") (concat)
    model = Model([movie_input, user_input, genres_input], result)
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
model2 = get_model2(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT, len(GENRES))

In [ ]:
plot_model(model2, show_shapes=True, show_layer_names=True)

In [ ]:
history2 = model2.fit([train_df['movieId'], train_df['userId'], train_df[GENRES]],
          train_df['rating'],
          batch_size=2048, 
          epochs=20,
          validation_split=0.1,
          shuffle=True)

In [ ]:
y_pred = model2.predict([test_df['movieId'], test_df['userId'], test_df[GENRES]]).reshape(-1)
y_true = test_df['rating'].values

rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Matrix-Factorization: {:.4f} RMSE'.format(rmse))

In [ ]:
plot_loss(history2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=200, stop_words='english', use_idf=True, ngram_range=(1,3))

tfidf_hybrid = tfidf.fit_transform(MOVIE_METADATA['overview'].str.lower())

In [ ]:
mapping = {id:i for i, id in enumerate(MOVIE_METADATA['movieId'])}
tfidf_hybrid_array = tfidf_hybrid.toarray()

train_tfidf = []

for id in train_df['movieId'].values:
    index = mapping[id]
    train_tfidf.append(tfidf_hybrid_array[index])
    
test_tfidf = []
for id in test_df['movieId'].values:
    index = mapping[id]
    test_tfidf.append(tfidf_hybrid_array[index])

In [ ]:
train_tfidf = np.vstack(train_tfidf)
test_tfidf = np.vstack(test_tfidf)
NUM_TFIDF = train_tfidf.shape[1]

In [ ]:
train_tfidf.shape, test_tfidf.shape

In [ ]:
def get_model3(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT, NUM_GENRES, NUM_TFIDF):
    from keras.layers import Input, Embedding, Dense, Dropout, Dot, Reshape, Concatenate
    from keras.models import Model

    movie_input = Input(shape=(1,), name='movie_id')

    movie_emb = Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_MOVIES, input_length=ROW_COUNT, name='movie_emb')(movie_input)

    movie_res = Reshape(target_shape=(EMBEDDING_SIZE,), name='movie_res')(movie_emb)

    user_input = Input(shape=(1,), name='user_id')

    user_emb = Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_USERS, input_length=ROW_COUNT, name='user_emb')(user_input)

    user_res = Reshape(target_shape=(EMBEDDING_SIZE,), name='user_res')(user_emb)
    
    merged = Dot(name = 'dot_product', normalize = True, axes = 1)([movie_res, user_res])

    #############################

    tfidf_input = Input(shape=(NUM_TFIDF,), name='tfidf', sparse=True)
    dense_tfidf_1 = Dense(70,name='FullyConnected-tfidf_1')(tfidf_input)
    dropout_tfidf_1 = Dropout(0.2,name='Dropout_tfidf_1')(dense_tfidf_1)

    dense_tfidf_2 = Dense(50,name='FullyConnected-tfidf_2')(dropout_tfidf_1)
    dropout_tfidf_2 = Dropout(0.2,name='Dropout_tfidf_2')(dense_tfidf_2)

    dense_tfidf_3 = Dense(20,name='FullyConnected-tfidf_3')(dropout_tfidf_2)
    dropout_tfidf_3 = Dropout(0.2,name='Dropout_tfidf_3')(dense_tfidf_3)

    dense_tfidf_4 = Dense(10,name='FullyConnected-tfidf_4', activation='relu')(dropout_tfidf_3)

    #############################
    genres_input = Input(shape=(NUM_GENRES,),name='genres')
    dense_gen_1 = Dense(70,name='FullyConnected-gen_1')(genres_input)
    dropout_gen_1 = Dropout(0.2,name='Dropout_gen_1')(dense_gen_1)

    dense_gen_2 = Dense(50,name='FullyConnected-gen_2')(dropout_gen_1)
    dropout_gen_2 = Dropout(0.2,name='Dropout_gen_2')(dense_gen_2)

    dense_gen_3 = Dense(20,name='FullyConnected-gen_3')(dropout_gen_2)
    dropout_gen_3 = Dropout(0.2,name='Dropout_gen_3')(dense_gen_3)

    dense_gen_4 = Dense(10,name='FullyConnected-gen_4', activation='relu')(dropout_gen_3)
    #############################

    merged = Dropout(0.2)(merged)
    
    dense_1 = Dense(70,name='FullyConnected-1')(merged)
    dropout_1 = Dropout(0.2,name='Dropout_1')(dense_1)

    dense_2 = Dense(50,name='FullyConnected-2')(dropout_1)
    dropout_2 = Dropout(0.2,name='Dropout_2')(dense_2)

    dense_3 = Dense(20,name='FullyConnected-3')(dropout_2)
    dropout_3 = Dropout(0.2,name='Dropout_3')(dense_3)

    dense_4 = Dense(10,name='FullyConnected-4', activation='relu')(dropout_3)

    concat = Concatenate()([dense_4, dense_gen_4, dense_tfidf_4])

    result = Dense(1, name='result', activation="relu") (concat)
    model = Model([movie_input, user_input, genres_input, tfidf_input], result)
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
model3 = get_model3(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT, len(GENRES), NUM_TFIDF)

In [ ]:
plot_model(model3, show_shapes=True, show_layer_names=True)

In [ ]:
history3 = model3.fit([train_df['movieId'], train_df['userId'], train_df[GENRES], train_tfidf],
          train_df['rating'],
          batch_size=2048, 
          epochs=20,
          validation_split=0.1,
          shuffle=True)

In [ ]:
y_pred = model3.predict([test_df['movieId'], test_df['userId'], test_df[GENRES], test_tfidf]).reshape(-1)
y_true = test_df['rating'].values

rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Matrix-Factorization: {:.4f} RMSE'.format(rmse))

In [ ]:
plot_loss(history3)

In [ ]:
def recommend_movies(user, model, topN = 5):

    type_layers = [x.__class__.__name__ for x in model.layers]
    len_input_layer = type_layers.count('InputLayer')

    if len_input_layer == 2:
        pred = model.predict([np.ones(len(UNIQUE_MOVIE_IDS))*user, UNIQUE_MOVIE_IDS]).reshape(-1)
        pred_value = np.sort(pred)[::-1]
        pred = np.argsort(pred)[::-1]

    
    if len_input_layer == 3:
        aux = MOVIE_METADATA.copy()

        for col in GENRES:
            aux[col] = MOVIE_METADATA["genres"].apply(lambda x: 1 if col in x else 0)
        
        pred = model.predict([np.ones(len(UNIQUE_MOVIE_IDS))*user, UNIQUE_MOVIE_IDS, aux[GENRES]]).reshape(-1)
        pred_value = np.sort(pred)[::-1]
        pred = np.argsort(pred)[::-1]

    if len_input_layer == 4:

        aux = MOVIE_METADATA.copy()

        for col in GENRES:
            aux[col] = MOVIE_METADATA["genres"].apply(lambda x: 1 if col in x else 0)
        
        mapping = {id:i for i, id in enumerate(MOVIE_METADATA['movieId'])}
        tfidf_hybrid_array = tfidf_hybrid.toarray()
            
        tfidf = []
        for id in MOVIE_METADATA['movieId'].values:
            index = mapping[id]
            tfidf.append(tfidf_hybrid_array[index])

        pred = model.predict([np.ones(len(UNIQUE_MOVIE_IDS))*user, UNIQUE_MOVIE_IDS, aux[GENRES], np.array(tfidf)]).reshape(-1)
        pred_value = np.sort(pred)[::-1]
        pred = np.argsort(pred)[::-1]
    
    movies_seen = list(ratings_df[ratings_df['userId']==user]['movieId'])
    
    cont, pos = 0, 0
    top_pred = []
    top_pred_value = []
    while cont < topN:
        if pred[pos] not in movies_seen:
            top_pred.append(pred[pos])
            top_pred_value.append(pred_value[pos])
            cont+=1
        pos+=1

    df_indices = pd.DataFrame({'movieId':top_pred, 'rating_pred':top_pred_value})
    return df_indices.merge(MOVIE_METADATA,on='movieId')

In [ ]:
recommend_movies(11, model1)

In [ ]:
recommend_movies(11, model2)

In [ ]:
recommend_movies(11, model3)

In [ ]:
def plot_losses(histories):
    import matplotlib.pyplot as plt

    plt.figure(figsize = (6,6))
    plt.suptitle('Training losses', fontsize=14, weight=600, color='#333d29')
    
    for i, history in enumerate(histories):
        training_loss = history.history['loss']
        test_loss = history.history['val_loss']

        epoch_count = range(1, len(training_loss) + 1)
        plt.subplot(len(histories),1,i+1)
        plt.grid()
        plt.plot(epoch_count, training_loss, '--', color='#06837f')
        plt.plot(epoch_count, test_loss, '-', color='#fed811')
        plt.legend(['Training Loss', 'Test Loss'])
        plt.title(f'Model {i+1}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
    plt.tight_layout(pad=0.7)

In [ ]:
plot_losses([history1, history2, history3])